In [70]:
import torch
import torch.nn as nn
import numpy as np

class ResNetRNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers, rnn_type = 'GRU'):
        super(ResNetRNN, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.conv_layers = nn.ModuleList()
        self.bn_layers = nn.ModuleList()

        for i in range(num_layers):
            self.conv_layers.append(nn.Conv1d(input_size, hidden_size, kernel_size=3, padding=1))
            self.bn_layers.append(nn.BatchNorm1d(hidden_size))
            input_size = hidden_size
            
        self.output_layer = nn.Conv1d(hidden_size, output_size, kernel_size=3, padding=1)
        
        self.rnn_type = rnn_type
        if rnn_type == 'GRU':
            self.rnn = nn.GRU(input_size=hidden_size, hidden_size=hidden_size, num_layers=num_layers)
        elif rnn_type == 'LSTM':
            self.rnn = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, num_layers=num_layers)
        else:
            raise ValueError("Invalid RNN type")
        
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        residual = x
        for i in range(self.num_layers):
            x = self.conv_layers[i](x)
            x = self.bn_layers[i](x)
            x = nn.ReLU()(x)
        x += residual
        x = nn.ReLU()(x)
        x = self.output_layer(x)
        
        x, _ = self.rnn(x)
        
        return x

    def loss(self, predictions, targets):
        return self.loss_fn(predictions, targets)


In [71]:
# Generate some example time series data
# This is just an example, and the data should be replaced with your actual time series data
sequence_length = 10
batch_size = 32
input_data = np.random.randn(batch_size, sequence_length, 3)

# Convert the data to a PyTorch tensor
input_data = torch.from_numpy(input_data).float()

target_data = np.random.randn(batch_size, sequence_length, 1)
target_data = torch.from_numpy(target_data).float()


In [72]:
input_data.shape, target_data.shape

(torch.Size([32, 10, 3]), torch.Size([32, 10, 1]))

In [73]:
# Instantiate the model
model = ResNetRNN(input_size=3, output_size=1, hidden_size=64, num_layers=1, rnn_type='GRU')

# Pass input data to the model
output = model(input_data)

# Compute the loss
loss = model.loss(output, target_data)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer.zero_grad()
loss.backward()
optimizer.step()


RuntimeError: Given groups=1, weight of size [64, 3, 3], expected input[32, 10, 3] to have 3 channels, but got 10 channels instead